In [1]:
import pandas as pd

df = pd.read_csv("../data/simpsons_dataset.csv")
df = df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131853 entries, 0 to 131852
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   raw_character_text  131853 non-null  object
 1   spoken_words        131853 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

corpus = list(df["spoken_words"].values)

stop_words = set(stopwords.words('english'))
w_n_lemmatizer = WordNetLemmatizer() 

corpus = [[w_n_lemmatizer.lemmatize(word) for word in word_tokenize(document.lower()) 
            if not word in stop_words and word.isalnum()] 
          for document in corpus]

corpus[0]

['actually',
 'little',
 'sometimes',
 'disease',
 'magazine',
 'news',
 'show',
 'natural',
 'think']

In [3]:
from gensim.models.fasttext import FastText

fastText = FastText()
fastText.build_vocab(corpus, progress_per=10000)
fastText.corpus_count

131853

In [4]:
fastText.train(corpus, 
               epochs=fastText.epochs,
               total_examples=fastText.corpus_count, 
               total_words=fastText.corpus_total_words)

In [5]:
fastText.init_sims(replace=True)

In [6]:
fastText.wv.most_similar(positive=["homer"])

[('knockahomer', 0.9014703631401062),
 ('marge', 0.8862709999084473),
 ('lisa', 0.8775784373283386),
 ('bart', 0.8767783045768738),
 ('homey', 0.8760688304901123),
 ('homeboy', 0.8532384037971497),
 ('sarge', 0.8470554947853088),
 ('barto', 0.8465577363967896),
 ('hometown', 0.8441286087036133),
 ('margie', 0.8399430513381958)]

In [7]:
fastText.wv.most_similar(positive=["marge"])

[('margie', 0.9583558440208435),
 ('maaarge', 0.9259381294250488),
 ('sarge', 0.8975951671600342),
 ('homer', 0.886271059513092),
 ('margarita', 0.883289635181427),
 ('lie', 0.8822088241577148),
 ('margaret', 0.8757448792457581),
 ('honey', 0.868972659111023),
 ('maggie', 0.8671963214874268),
 ('marie', 0.8648054003715515)]

In [8]:
fastText.wv.most_similar(positive=["bart"])

[('barto', 0.9328980445861816),
 ('lisa', 0.924902081489563),
 ('homer', 0.8767783641815186),
 ('bartholomew', 0.8484803438186646),
 ('bartron', 0.8439726829528809),
 ('maggie', 0.8404480218887329),
 ('knockahomer', 0.8402232527732849),
 ('barf', 0.8358963131904602),
 ('homework', 0.8257026672363281),
 ('visa', 0.8242322206497192)]

In [9]:
fastText.wv.similarity("homer", 'bart')

0.87677836

In [10]:
fastText.wv.doesnt_match(['jimbo', 'milhouse', 'kearney'])

C:\Users\opell.DESKTOP-UEQ8DPV\anaconda3\envs\E01\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'milhouse'

In [11]:
fastText.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

[('mansion', 0.9084514379501343),
 ('performer', 0.8903785943984985),
 ('decision', 0.8900672197341919)]

In [12]:
fastText.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

[('barto', 0.8755788803100586),
 ('lisa', 0.8724665641784668),
 ('homework', 0.8391784429550171)]